This Notebook is for analyzing temperature data from RCP8.5 projections.

In [1]:
#initiate Model
using Mimi
using CSV
using DataFrames
using Plots

After initiating the packages we will use we need to open and read the data.

In [2]:

tempdf = CSV.read("projections_temperature_RCP85_sneasybrick.csv", DataFrame)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,-0.0315825,0.0055617,-0.0237161,-0.0271995,-0.186869,-0.262602,-0.00582027,-0.108196,-0.00497589,0.157632,0.0329254,-0.0117298,-0.0780873,-0.158684,0.0965524,-0.33372,0.0131186,-0.183965,-0.123175,0.181937,-0.226657,-0.255351,-0.23516,-0.0723387,0.00138053,-0.020984,0.159412,-0.298994,-0.185455,-0.00364868,-0.260957,-0.150862,-0.0757289,-0.224968,-0.178638,-0.0192044,-0.0178433,0.00716368,0.114131,-0.054573,0.125365,-0.0365097,0.025781,-0.37606,-0.134656,-0.169177,-0.0177943,-0.203273,-0.0214763,0.0443029,-0.0353914,-0.144281,0.158883,-0.175173,-0.00124031,-0.143026,-0.0303204,-0.0770734,-0.00845246,-0.15505,0.0643635,-0.0857155,0.0712519,-0.0385653,0.04495,-0.0963797,0.149852,0.236969,-0.0394935,0.108053,0.00438547,0.268021,-0.169802,-0.217696,-0.0334035,-0.0432714,-0.0518406,0.0396081,-0.0301235,-0.232362,0.131617,-0.146289,0.0584418,0.0144428,0.00472612,-0.191595,-0.0229382,-0.0502981,0.0381103,0.0137722,0.0951725,-0.119384,-0.0589347,-0.255106,-0.00975045,-0.0512635,-0.120994,0.0380591,0.0703251,0.145699,⋯
2,0.0542699,-0.141403,0.000958853,-0.111827,-0.133155,-0.0388204,-0.230095,-0.0351384,0.0200956,0.0479486,-0.06667,0.180547,-0.142886,-0.115237,-0.144817,-0.0434511,-0.0865206,0.103128,-0.0412958,-0.172783,0.00174384,-0.103631,-0.156265,-0.180665,-0.137266,0.110538,-0.0838044,-0.152377,0.0188842,0.00912538,0.0490107,0.245256,0.0769207,0.0191031,-0.128508,0.0252513,0.0987948,-0.0441999,0.183156,0.0299245,0.177754,0.1572,0.0968056,0.0116079,-0.089561,0.184951,-0.106348,-0.0680507,0.0620432,0.0932063,0.176312,0.124057,0.075956,0.0863115,-0.0720135,-0.16907,-0.0052618,-0.19235,-0.00210637,0.180595,0.0186096,-0.00475638,-0.018084,-0.133507,0.0923664,0.191984,-0.0711732,0.0705638,-0.267502,-0.256421,0.0443645,0.0582318,0.224932,-0.404083,-0.0861219,0.101724,0.161876,-0.0884815,-0.00641207,-0.16278,-0.102208,0.0288759,0.326483,0.0792877,0.13421,-0.0930158,0.11344,0.0218009,0.0523426,0.109139,0.0837071,0.075803,0.158266,-0.138761,0.218152,0.384112,-0.0542485,-0.0158553,0.23462,0.0332916,⋯
3,0.010358,-0.109326,0.0403634,0.049863,0.114645,-0.100423,-0.0399549,-0.160802,0.0433712,0.0209195,-0.0751727,-0.169811,-0.0123062,-0.0178168,0.160433,-0.259128,0.302603,0.142245,0.0473617,-0.0246202,-0.0166267,-0.0899817,-0.00364028,0.100045,-0.16031,0.0281432,0.0888022,0.0317305,0.263121,-0.073367,-0.000590004,0.0494175,0.164877,-0.208642,0.040659,-0.0420332,0.055178,-0.0218358,0.119182,-0.0431615,0.131381,-0.133366,0.028709,-0.191209,-0.0029919,-0.00651213,-0.255772,-0.13079,-0.189336,0.207531,-0.240183,0.0692146,0.0261819,-0.0896661,0.122287,0.214478,-0.0906552,-0.0496562,0.253651,0.222455,-0.0166875,0.106031,0.0306571,-0.0315721,0.00292837,-0.177049,-0.146752,-0.0121592,0.0334891,-0.273455,-0.128669,0.0814376,-0.103244,-0.0637299,0.0781922,0.058477,0.120943,0

Next, we index through each temperature of the first 50 years for each trial and collect them.

In [45]:
# Set the index and array to solve for normalized value
sarray = []
i = 0

# Add each value to a sum
for r in collect(1:50)
    for c in collect(1:10000)
        push!(sarray, tempdf[r, c])
        i += 1
    end
end
sarray[1]

-0.03158245263471624

Next we find the mean.

In [48]:
# Collect the sum
sum = 0
for t in collect(1:i)
    sum += sarray[t]
end
# Divide by the number of terms(i)
mean = sum / i


-0.03028032912540104